<a href="https://colab.research.google.com/github/adigenova/uohpmd/blob/main/code/MPI_I.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!apt install mpich

Reading package lists... Done
Building dependency tree... Done
Reading state information... Done
The following additional packages will be installed:
  hwloc-nox libmpich-dev libmpich12 libslurm37
Suggested packages:
  mpich-doc
The following NEW packages will be installed:
  hwloc-nox libmpich-dev libmpich12 libslurm37 mpich
0 upgraded, 5 newly installed, 0 to remove and 49 not upgraded.
Need to get 14.2 MB of archives.
After this operation, 102 MB of additional disk space will be used.
Get:1 http://archive.ubuntu.com/ubuntu jammy/universe amd64 libslurm37 amd64 21.08.5-2ubuntu1 [542 kB]
Get:2 http://archive.ubuntu.com/ubuntu jammy-updates/universe amd64 hwloc-nox amd64 2.7.0-2ubuntu1 [205 kB]
Get:3 http://archive.ubuntu.com/ubuntu jammy/universe amd64 libmpich12 amd64 4.0-3 [5,866 kB]
Get:4 http://archive.ubuntu.com/ubuntu jammy/universe amd64 mpich amd64 4.0-3 [197 kB]
Get:5 http://archive.ubuntu.com/ubuntu jammy/universe amd64 libmpich-dev amd64 4.0-3 [7,375 kB]
Fetched 14.2 MB in 

In [11]:
%%writefile mpi_holamundo.c
// An intro MPI hello world program that uses MPI_Init, MPI_Comm_size,
// MPI_Comm_rank, MPI_Finalize, and MPI_Get_processor_name.
//
#include <mpi.h>
#include <stdio.h>

int main(int argc, char** argv) {
  // Initialize the MPI environment. The two arguments to MPI Init are not
  // currently used by MPI implementations, but are there in case future
  // implementations might need the arguments.
  MPI_Init(NULL, NULL);

  // Get the number of processes
  int world_size;
  MPI_Comm_size(MPI_COMM_WORLD, &world_size);

  // Get the rank of the process
  int world_rank;
  MPI_Comm_rank(MPI_COMM_WORLD, &world_rank);

  // Get the name of the processor
  char processor_name[MPI_MAX_PROCESSOR_NAME];
  int name_len;
  MPI_Get_processor_name(processor_name, &name_len);
  if (world_rank > 5){
  printf("Hola mundo desde procesador %s, ranking %d de  %d procesadores\n",
         processor_name, world_rank, world_size);
  }else{
    printf("castigado ranking %d\n",world_rank);
  }
  // Finalize the MPI environment. No more MPI calls can be made after this
  MPI_Finalize();
}

Overwriting mpi_holamundo.c


In [12]:
! mpicc -o mpi_holamundo mpi_holamundo.c

In [13]:
! mpirun --oversubscribe --allow-run-as-root -np 10 ./mpi_holamundo

castigado ranking 4
Hola mundo desde procesador 6fbaed727665, ranking 8 de  10 procesadores
Hola mundo desde procesador 6fbaed727665, ranking 6 de  10 procesadores
castigado ranking 2
castigado ranking 5
castigado ranking 3
Hola mundo desde procesador 6fbaed727665, ranking 7 de  10 procesadores
castigado ranking 0
castigado ranking 1
Hola mundo desde procesador 6fbaed727665, ranking 9 de  10 procesadores


In [15]:
%%writefile mpi_send_recv.c

#include <mpi.h>
#include <stdio.h>
#include <stdlib.h>

int main(int argc, char** argv) {
  // Initialize the MPI environment
  MPI_Init(NULL, NULL);
  // Find out rank, size
  int world_rank;
  MPI_Comm_rank(MPI_COMM_WORLD, &world_rank);
  int world_size;
  MPI_Comm_size(MPI_COMM_WORLD, &world_size);

  // Usamos almenos dos procesos para ejecutar esta tarea
  if (world_size < 2) {
    fprintf(stderr, "el numero de procesos debe ser > 1 for %s\n", argv[0]);
    MPI_Abort(MPI_COMM_WORLD, 1);
  }

  int number;
  if (world_rank == 0) {
    //Si el ranking es 0, inicializamos el numero en 10 y lo enviamos al proceso 1
    number = 10;
    MPI_Send(
      /* dato         = */ &number,
      /* cantidad        = */ 1,
      /* typo     = */ MPI_INT,
      /* destino  = */ 1,
      /* tag          = */ 0,
      /* comunicador = */ MPI_COMM_WORLD);
  } else if (world_rank == 1) {
    MPI_Recv(
      /* dato         = */ &number,
      /* cantidad        = */ 1,
      /* tipo     = */ MPI_INT,
      /* origen       = */ 0,
      /* tag          = */ 0,
      /* comunicador = */ MPI_COMM_WORLD,
      /* stado       = */ MPI_STATUS_IGNORE);
    printf("Proceso 1 recibio el numero %d desde el proceso 0\n", number);
    for(int i=0; i< number; i++){
        printf("Realizando iteracion %d de %d\n", i, number);
    }
  }
  MPI_Finalize();
}

Overwriting mpi_send_recv.c


In [16]:
! mpicc -o mpi_send_recv mpi_send_recv.c

In [18]:
! mpirun --oversubscribe --allow-run-as-root -np 4 ./mpi_send_recv

Proceso 1 recibio el numero 10 desde el proceso 0
Realizando iteracion 0 de 10
Realizando iteracion 1 de 10
Realizando iteracion 2 de 10
Realizando iteracion 3 de 10
Realizando iteracion 4 de 10
Realizando iteracion 5 de 10
Realizando iteracion 6 de 10
Realizando iteracion 7 de 10
Realizando iteracion 8 de 10
Realizando iteracion 9 de 10


In [ ]:
%%writefile mpi_ping_poing.c

#include <mpi.h>
#include <stdio.h>
#include <stdlib.h>

int main(int argc, char** argv) {
  const int PING_PONG_LIMIT = 10;

  // Initialize the MPI environment
  MPI_Init(NULL, NULL);
  // Find out rank, size
  int world_rank;
  MPI_Comm_rank(MPI_COMM_WORLD, &world_rank);
  int world_size;
  MPI_Comm_size(MPI_COMM_WORLD, &world_size);

  // usamos dos procesos para esta tarea
  if (world_size != 2) {
    fprintf(stderr, " utilizamos dos procesos %s\n", argv[0]);
    MPI_Abort(MPI_COMM_WORLD, 1);
  }

  int ping_pong_count = 0;
  int partner_rank = (world_rank + 1) % 2;
  while (ping_pong_count < PING_PONG_LIMIT) {
    if (world_rank == ping_pong_count % 2) {
      // imcrementamos el valor antes de enviarlo
      ping_pong_count++;
      MPI_Send(&ping_pong_count, 1, MPI_INT, partner_rank, 0, MPI_COMM_WORLD);
      printf("%d enviamos y incrementamos el contador ping_pong_count %d to %d\n",
             world_rank, ping_pong_count, partner_rank);
    } else {
      MPI_Recv(&ping_pong_count, 1, MPI_INT, partner_rank, 0, MPI_COMM_WORLD,
               MPI_STATUS_IGNORE);
      printf("%d recibido ping_pong_count %d desde %d\n",
             world_rank, ping_pong_count, partner_rank);
    }
  }
  MPI_Finalize();
}

Writing mpi_ping_poing.c


In [ ]:
! mpicc -o mpi_ping_poing mpi_ping_poing.c

In [ ]:
! mpirun --allow-run-as-root -np 2 ./mpi_ping_poing

0 enviamos y incrementamos el contador ping_pong_count 1 to 1
1 recibido ping_pong_count 1 desde 0
1 enviamos y incrementamos el contador ping_pong_count 2 to 0
0 recibido ping_pong_count 2 desde 1
0 enviamos y incrementamos el contador ping_pong_count 3 to 1
1 recibido ping_pong_count 3 desde 0
1 enviamos y incrementamos el contador ping_pong_count 4 to 0
0 recibido ping_pong_count 4 desde 1
0 enviamos y incrementamos el contador ping_pong_count 5 to 1
1 recibido ping_pong_count 5 desde 0
1 enviamos y incrementamos el contador ping_pong_count 6 to 0
0 recibido ping_pong_count 6 desde 1
0 enviamos y incrementamos el contador ping_pong_count 7 to 1
1 recibido ping_pong_count 7 desde 0
1 enviamos y incrementamos el contador ping_pong_count 8 to 0
0 recibido ping_pong_count 8 desde 1
0 enviamos y incrementamos el contador ping_pong_count 9 to 1
1 recibido ping_pong_count 9 desde 0
1 enviamos y incrementamos el contador ping_pong_count 10 to 0
0 recibido ping_pong_count 10 desde 1
